This dataset contains a provider-level summary of inpatient discharges, average charges, and average Medicare payments for the top 100 Diagnosis-Related Groups (DRGs).

filename: CMS_Medicare_OpenSource_Data.csv

1. Which types of DRGs ("DRG.Definition") account for most of the patient population across all facilities?

2. For each facility ("Provider.Name"), which DRGs form the largest portion of the patient population?

3. Calculate the average Medicare payments per DRG per facility.

4. Is there a way to group facilities in a logical way?

Please provide any code (in either Python or R) that you used.  If you have any figures you'd like to show (though not at all required), feel free to include them in a separate file or inline in a notebook/markdown.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

Read data into a dataframe.

In [4]:
df = pd.read_csv('CMS_Medicare_OpenSource_Data.txt')
df.sample(5)

DRG Definition  Provider Id  \
19854         189 - PULMONARY EDEMA & RESPIRATORY FAILURE       260017   
148262             812 - RED BLOOD CELL DISORDERS W/O MCC       140176   
58159   286 - CIRCULATORY DISORDERS EXCEPT AMI, W CARD...       230017   
160738   918 - POISONING & TOXIC EFFECTS OF DRUGS W/O MCC       320001   
85174      314 - OTHER CIRCULATORY SYSTEM DIAGNOSES W MCC       230047   

                                      Provider Name  Provider Street Address  \
19854         PHELPS COUNTY REGIONAL MEDICAL CENTER           1000 W 10TH ST   
148262  CENTEGRA HEALTH SYSTEM - WOODSTOCK HOSPITAL           3701 DOTY ROAD   
58159                    BRONSON METHODIST HOSPITAL          601 JOHN STREET   
160738                                 UNM HOSPITAL  2211 LOMAS BOULEVARD NE   
85174                    HENRY FORD MACOMB HOSPITAL   15855 NINETEEN MILE RD   

          Provider City Provider State  Provider Zip Code  \
19854             ROLLA             MO              65401   
148262        WOODSTOCK             IL              60098   
58159         KALAMAZOO             MI              49007   
160738      ALBUQUERQUE             NM              87106   
85174   CLINTON TOWNSHI             MI              48038   

       Hospital Referral Region Description   Total Discharges   \
19854                        MO - St. Louis                  32   
148262                        IL - Rockford                  35   
58159                        MI - Kalamazoo                  15   
160738                     NM - Albuquerque                  28   
85174                          MI - Detroit                  59   

        Average Covered Charges   Average Total Payments   \
19854                  $35398.06                 $9901.50   
148262                 $19640.08                 $5153.22   
58159                  $42753.93                $15898.73   
160738                 $12021.89                 $6287.35   
85174                  $23344.47                $10980.45   

       Average Medicare Payments  
19854                   $8947.40  
148262                  $3876.20  
58159                  $15070.80  
160738                  $5482.21  
85174                  $10321.37

Check for null values.

In [5]:
df.isnull().sum()

DRG Definition                          0
Provider Id                             0
Provider Name                           0
Provider Street Address                 0
Provider City                           0
Provider State                          0
Provider Zip Code                       0
Hospital Referral Region Description    0
 Total Discharges                       0
 Average Covered Charges                0
 Average Total Payments                 0
Average Medicare Payments               0
dtype: int64

No null values so data is clean. Rename the columns for ease of use.

In [6]:
df.rename(columns={'DRG Definition': 'DRG', 
                   'Provider Id': 'Prov_Id', 
                   'Provider Name': 'Prov_Name', 
                   'Provider Street Address': 'Prov_Address', 
                   'Provider City': 'Prov_City',
                   'Provider State': 'Prov_State',
                   'Provider Zip Code': 'Prov_Zip',
                   'Hospital Referral Region Description': 'Hospital_Region',
                   ' Total Discharges ': 'Reg_Total_Discharges',
                   ' Average Covered Charges ': 'Reg_Avg_Covered_Chgs',
                   ' Average Total Payments ': 'Reg_Avg_Total_Pmts',
                   'Average Medicare Payments': 'Avg_Medicare_Pmts',
                  },
          inplace=True)

In [7]:
df.columns

Index(['DRG', 'Prov_Id', 'Prov_Name', 'Prov_Address', 'Prov_City',
       'Prov_State', 'Prov_Zip', 'Hospital_Region', 'Reg_Total_Discharges',
       'Reg_Avg_Covered_Chgs', 'Reg_Avg_Total_Pmts', 'Avg_Medicare_Pmts'],
      dtype='object')

In [8]:
df[:3].T

0  \
DRG                   039 - EXTRACRANIAL PROCEDURES W/O CC/MCC   
Prov_Id                                                  10001   
Prov_Name                     SOUTHEAST ALABAMA MEDICAL CENTER   
Prov_Address                            1108 ROSS CLARK CIRCLE   
Prov_City                                               DOTHAN   
Prov_State                                                  AL   
Prov_Zip                                                 36301   
Hospital_Region                                    AL - Dothan   
Reg_Total_Discharges                                        91   
Reg_Avg_Covered_Chgs                                 $32963.07   
Reg_Avg_Total_Pmts                                    $5777.24   
Avg_Medicare_Pmts                                     $4763.73   

                                                             1  \
DRG                   039 - EXTRACRANIAL PROCEDURES W/O CC/MCC   
Prov_Id                                                  10005   
Prov_Name                        MARSHALL MEDICAL CENTER SOUTH   
Prov_Address                        2505 U S HIGHWAY 431 NORTH   
Prov_City                                                 BOAZ   
Prov_State                                                  AL   
Prov_Zip                                                 35957   
Hospital_Region                                AL - Birmingham   
Reg_Total_Discharges                                        14   
Reg_Avg_Covered_Chgs                                 $15131.85   
Reg_Avg_Total_Pmts                                    $5787.57   
Avg_Medicare_Pmts                                     $4976.71   

                                                             2  
DRG                   039 - EXTRACRANIAL PROCEDURES W/O CC/MCC  
Prov_Id                                                  10006  
Prov_Name                       ELIZA COFFEE MEMORIAL HOSPITAL  
Prov_Address                                205 MARENGO STREET  
Prov_City                                             FLORENCE  
Prov_State                                                  AL  
Prov_Zip                                                 35631  
Hospital_Region                                AL - Birmingham  
Reg_Total_Discharges                                        24  
Reg_Avg_Covered_Chgs                                 $37560.37  
Reg_Avg_Total_Pmts                                    $5434.95  
Avg_Medicare_Pmts                                     $4453.79

# 1. Which types of DRGs ("DRG.Definition") account for most of the patient population across all facilities?

How many unique DRG values are there?

In [9]:
df['DRG'].count()

163065

What are the value counts of the top ten DRG Definitions?

In [10]:
df['DRG'].value_counts()[:10]

194 - SIMPLE PNEUMONIA & PLEURISY W CC                                      3023
690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                             2989
292 - HEART FAILURE & SHOCK W CC                                            2953
392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC                2950
641 - MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES W/O MCC    2899
871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC                    2812
603 - CELLULITIS W/O MCC                                                    2807
470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC    2750
191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC                            2720
190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC                           2713
Name: DRG, dtype: int64

But we want the top of the patient population across all facilities, so need to count the Reg_Total_Discharges.

In [11]:
DRG_series = df.groupby(['DRG'])['Reg_Total_Discharges'].sum().sort_values(ascending=False)
DRG_df = pd.DataFrame(DRG_series).reset_index()
DRG_df.head()

DRG  Reg_Total_Discharges
0  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...                427207
1  871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...                319072
2  392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...                244854
3                   292 - HEART FAILURE & SHOCK W CC                222038
4    690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                206695

The question did not specify what percentage of cases are requested, so I will interpret most to mean 50% of all the data.

The rows are sorted by highest count. Find how many of the top rows of DRG Definitions values comp`rise 50% of the whole dataset.

In [12]:
DRG_df.Reg_Total_Discharges.sum()

6975318

Calculate what 50% of total sum is:

In [13]:
.5 * DRG_df.Reg_Total_Discharges.sum()

3487659.0

Create a dataframe with just the total discharge for each DRG sorted.

In [14]:
Top_DRG_df = DRG_df.sort_values('Reg_Total_Discharges',ascending=False)

In [15]:
Top_DRG_df.head(5)

DRG  Reg_Total_Discharges
0  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...                427207
1  871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...                319072
2  392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...                244854
3                   292 - HEART FAILURE & SHOCK W CC                222038
4    690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                206695

Calculate the cumulative sum of the total discharges.

In [16]:
DRG_df.sort_values('Reg_Total_Discharges',ascending=False).cumsum().head(5)

DRG Reg_Total_Discharges
0  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...               427207
1  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...               746279
2  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...               991133
3  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...              1213171
4  470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...              1419866

Add a column for the cumulative sum.

In [17]:
DRG_df['cumsum'] = DRG_df.sort_values('Reg_Total_Discharges',ascending=False).cumsum()['Reg_Total_Discharges']

In [18]:
DRG_df['cum_perc'] = DRG_df['cumsum']/ DRG_df.Reg_Total_Discharges.sum()

Print the DRGs that account for 50% of the DRGs across all facilities.

In [19]:
DRG_df.head(20)

DRG  Reg_Total_Discharges  \
0   470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...                427207   
1   871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...                319072   
2   392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...                244854   
3                    292 - HEART FAILURE & SHOCK W CC                222038   
4     690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                206695   
5              194 - SIMPLE PNEUMONIA & PLEURISY W CC                198390   
6                   291 - HEART FAILURE & SHOCK W MCC                185599   
7   641 - MISC DISORDERS OF NUTRITION,METABOLISM,F...                153660   
8                            683 - RENAL FAILURE W CC                150444   
9   190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC                149677   
10   191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC                148491   
11                           312 - SYNCOPE & COLLAPSE                141918   
12                           603 - CELLULITIS W/O MCC                140894   
13                         378 - G.I. HEMORRHAGE W CC                138678   
14                                   313 - CHEST PAIN                131079   
15            193 - SIMPLE PNEUMONIA & PLEURISY W MCC                127989   
16  287 - CIRCULATORY DISORDERS EXCEPT AMI, W CARD...                115920   
17  192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/...                114790   
18  310 - CARDIAC ARRHYTHMIA & CONDUCTION DISORDER...                113454   
19  872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...                112430   

     cumsum   cum_perc  
0    427207  0.0612455  
1    746279   0.106989  
2    991133   0.142091  
3   1213171   0.173923  
4   1419866   0.203556  
5   1618256   0.231997  
6   1803855   0.258605  
7   1957515   0.280635  
8   2107959   0.302203  
9   2257636   0.323661  
10  2406127   0.344949  
11  2548045   0.365294  
12  2688939   0.385493  
13  2827617   0.405375  
14  2958696   0.424166  
15  3086685   0.442515  
16  3202605   0.459134  
17  3317395   0.475591  
18  3430849   0.491856  
19  3543279   0.507974

# 2. For each facility ("Provider.Name"), which DRGs form the largest portion of the patient population?

Start by priming a list of the information for the first hospital. 

In [20]:
# For the SE AL Med Ctr, select three columns and sort by the # of discharges.
Reg_Tot_Discharges_df = df.loc[df['Prov_Name'] == 'SOUTHEAST ALABAMA MEDICAL CENTER', 
       ['Prov_Name', 'Reg_Total_Discharges', 'DRG']] \
  .sort_values(by='Reg_Total_Discharges', ascending=False)

# Then add a column for the cumulative sum of discharges.
Reg_Tot_Discharges_df['cumsum'] = Reg_Tot_Discharges_df['Reg_Total_Discharges'].cumsum()

# Now add a column for the percentage of all discharges at the hospital.
Reg_Tot_Discharges_df['Dischg_perc'] = Reg_Tot_Discharges_df['cumsum'] / Reg_Tot_Discharges_df['Reg_Total_Discharges'].sum()

# Lastly, add that information into a dataframe that will collect the info for all hospitals.
hosp_df = pd.DataFrame(Reg_Tot_Discharges_df[['Prov_Name', 'Dischg_perc', 'DRG']].head(1))

Now that we've primed our hosp_df dataframe, let's loop through all of the hospitals. To run this for all of the hospitals would take quite a while, so I'm running it for only the first 50 hospitals. 

In [21]:
for hospital in df.Prov_Name.values.tolist()[1:50]:
    # For each hospital, select three columns and sort by the # of discharges.
    Reg_Tot_Discharges_df = df.loc[df['Prov_Name'] == hospital, 
           ['Prov_Name', 'Reg_Total_Discharges', 'DRG']] \
      .sort_values(by='Reg_Total_Discharges', ascending=False)

    # Then add a column for the cumulative sum of discharges.
    Reg_Tot_Discharges_df['cumsum'] = Reg_Tot_Discharges_df['Reg_Total_Discharges'].cumsum()

    # Now add a column for the percentage of all discharges at the hospital.
    Reg_Tot_Discharges_df['Dischg_perc'] = Reg_Tot_Discharges_df['cumsum'] / Reg_Tot_Discharges_df['Reg_Total_Discharges'].sum()

    # Lastly, append that information to our hosp_df dataframe.
    hosp_df = hosp_df.append(Reg_Tot_Discharges_df[['Prov_Name', 'Dischg_perc', 'DRG']].head(1))

# reset the index and print the list
hosp_df.reset_index(inplace=True)
hosp_df

index                                    Prov_Name  Dischg_perc  \
0   157559             SOUTHEAST ALABAMA MEDICAL CENTER     0.054579   
1    20890                MARSHALL MEDICAL CENTER SOUTH     0.059901   
2   111270               ELIZA COFFEE MEMORIAL HOSPITAL     0.054089   
3   111273                            ST VINCENT'S EAST     0.078162   
4    58667                SHELBY BAPTIST MEDICAL CENTER     0.047241   
5   111278                 BAPTIST MEDICAL CENTER SOUTH     0.058428   
6   111281          EAST ALABAMA MEDICAL CENTER AND SNF     0.073159   
7   152232               UNIVERSITY OF ALABAMA HOSPITAL     0.034005   
8   111286                          HUNTSVILLE HOSPITAL     0.078518   
9   111287              GADSDEN REGIONAL MEDICAL CENTER     0.059982   
10  152241            RIVERVIEW REGIONAL MEDICAL CENTER     0.057235   
11  111291                             FLOWERS HOSPITAL     0.087714   
12  111292                      ST VINCENT'S BIRMINGHAM     0.122383   
13  111295        NORTHEAST ALABAMA REGIONAL MED CENTER     0.043264   
14   58682        SOUTH BALDWIN REGIONAL MEDICAL CENTER     0.053476   
15  150834                     DECATUR GENERAL HOSPITAL     0.098129   
16  111301                          PROVIDENCE HOSPITAL     0.042799   
17  101775                D C H REGIONAL MEDICAL CENTER     0.040278   
18  111303                              THOMAS HOSPITAL     0.151633   
19   46833             BAPTIST MEDICAL CENTER-PRINCETON     0.047720   
20  111305                       TRINITY MEDICAL CENTER     0.066762   
21  101785                             MOBILE INFIRMARY     0.043630   
22  111312                     BROOKWOOD MEDICAL CENTER     0.080175   
23  111325                     ALASKA REGIONAL HOSPITAL     0.298003   
24  111328         BANNER GOOD SAMARITAN MEDICAL CENTER     0.062110   
25  111329                        TUCSON MEDICAL CENTER     0.160779   
26  111330                  VERDE VALLEY MEDICAL CENTER     0.120930   
27  152295                CARONDELET ST  MARYS HOSPITAL     0.057307   
28  111332               CARONDELET ST JOSEPHS HOSPITAL     0.063931   
29  111333              YAVAPAI REGIONAL MEDICAL CENTER     0.111376   
30  111334                 YUMA REGIONAL MEDICAL CENTER     0.050863   
31  111337                     FLAGSTAFF MEDICAL CENTER     0.132887   
32  109044      ST JOSEPH'S HOSPITAL AND MEDICAL CENTER     0.067556   
33   42630                     PHOENIX BAPTIST HOSPITAL     0.102883   
34  111343  SCOTTSDALE HEALTHCARE-OSBORN MEDICAL CENTER     0.107973   
35  111346                BANNER BOSWELL MEDICAL CENTER     0.074098   
36  111347    SUMMIT HEALTHCARE REGIONAL MEDICAL CENTER     0.191987   
37  152315                 BANNER DESERT MEDICAL CENTER     0.037721   
38  111351               HAVASU REGIONAL MEDICAL CENTER     0.124766   
39  111355    SCOTTSDALE HEALTHCARE-SHEA MEDICAL CENTER     0.173549   
40  101846            BANNER THUNDERBIRD MEDICAL CENTER     0.050660   
41  152323          JOHN C LINCOLN DEER VALLEY HOSPITAL     0.069079   
42  111359             BANNER DEL E WEBB MEDICAL CENTER     0.082064   
43  111360                           ARROWHEAD HOSPITAL     0.115324   
44   46869      CARONDELET HEART AND VASCULAR INSTITUTE     0.139219   
45  111361         WESTERN ARIZONA REGIONAL MEDICAL CTR     0.084948   
46  111362                         MAYO CLINIC HOSPITAL     0.112894   
47   63156                        BANNER HEART HOSPITAL     0.098780   
48  111379   WASHINGTON REGIONAL MED CTR AT NORTH HILLS     0.085100   
49  111380          ST VINCENT INFIRMARY MEDICAL CENTER     0.085939   

                                                  DRG  
0                                     885 - PSYCHOSES  
1   190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC  
2   470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...  
3   470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...  
4   287 - CIRCULATORY DISORDERS EXCEPT AMI, W CARD..

In [23]:
hosp_df.Dischg_perc.sum()

4.39598973107674

# 3. Calculate the average Medicare payments per DRG per facility.

Check the types of each column.

In [20]:
df.dtypes

DRG                     object
Prov_Id                  int64
Prov_Name               object
Prov_Address            object
Prov_City               object
Prov_State              object
Prov_Zip                 int64
Hospital_Region         object
Reg_Total_Discharges     int64
Reg_Avg_Covered_Chgs    object
Reg_Avg_Total_Pmts      object
Avg_Medicare_Pmts       object
dtype: object

Drop the $ from the average medicare payments field and then convert it to a float.

In [21]:
df['Avg_Medicare_Pmts'] = df['Avg_Medicare_Pmts'].str \
                                                 .replace('$', '') \
                                                 .astype(float)
df['Avg_Medicare_Pmts'].head()

0    4763.73
1    4976.71
2    4453.79
3    4129.16
4    4851.44
Name: Avg_Medicare_Pmts, dtype: float64

Now group the rows by provider name and the DRG and then print the average of the Avg_Medicare_Pmts.

In [22]:
df.groupby(['Prov_Name', 'DRG'], as_index=False)['Avg_Medicare_Pmts'].mean()

Prov_Name  \
0                       ABBEVILLE GENERAL HOSPITAL   
1                       ABBEVILLE GENERAL HOSPITAL   
2                       ABBEVILLE GENERAL HOSPITAL   
3                       ABBEVILLE GENERAL HOSPITAL   
4                       ABBEVILLE GENERAL HOSPITAL   
5                       ABBEVILLE GENERAL HOSPITAL   
6                       ABBEVILLE GENERAL HOSPITAL   
7                       ABBEVILLE GENERAL HOSPITAL   
8                       ABBEVILLE GENERAL HOSPITAL   
9                       ABBEVILLE GENERAL HOSPITAL   
10                      ABBEVILLE GENERAL HOSPITAL   
11                      ABBEVILLE GENERAL HOSPITAL   
12                      ABBEVILLE GENERAL HOSPITAL   
13                      ABBEVILLE GENERAL HOSPITAL   
14                      ABBEVILLE GENERAL HOSPITAL   
15                      ABBEVILLE GENERAL HOSPITAL   
16                      ABBEVILLE GENERAL HOSPITAL   
17                      ABBEVILLE GENERAL HOSPITAL   
18                    ABBOTT NORTHWESTERN HOSPITAL   
19                    ABBOTT NORTHWESTERN HOSPITAL   
20                    ABBOTT NORTHWESTERN HOSPITAL   
21                    ABBOTT NORTHWESTERN HOSPITAL   
22                    ABBOTT NORTHWESTERN HOSPITAL   
23                    ABBOTT NORTHWESTERN HOSPITAL   
24                    ABBOTT NORTHWESTERN HOSPITAL   
25                    ABBOTT NORTHWESTERN HOSPITAL   
26                    ABBOTT NORTHWESTERN HOSPITAL   
27                    ABBOTT NORTHWESTERN HOSPITAL   
28                    ABBOTT NORTHWESTERN HOSPITAL   
29                    ABBOTT NORTHWESTERN HOSPITAL   
...                                            ...   
157343                YUMA REGIONAL MEDICAL CENTER   
157344                YUMA REGIONAL MEDICAL CENTER   
157345                YUMA REGIONAL MEDICAL CENTER   
157346                YUMA REGIONAL MEDICAL CENTER   
157347                YUMA REGIONAL MEDICAL CENTER   
157348                YUMA REGIONAL MEDICAL CENTER   
157349                YUMA REGIONAL MEDICAL CENTER   
157350                YUMA REGIONAL MEDICAL CENTER   
157351                YUMA REGIONAL MEDICAL CENTER   
157352                YUMA REGIONAL MEDICAL CENTER   
157353                YUMA REGIONAL MEDICAL CENTER   
157354                YUMA REGIONAL MEDICAL CENTER   
157355                YUMA REGIONAL MEDICAL CENTER   
157356                YUMA REGIONAL MEDICAL CENTER   
157357                YUMA REGIONAL MEDICAL CENTER   
157358                YUMA REGIONAL MEDICAL CENTER   
157359                YUMA REGIONAL MEDICAL CENTER   
157360                YUMA REGIONAL MEDICAL CENTER   
157361                YUMA REGIONAL MEDICAL CENTER   
157362                YUMA REGIONAL MEDICAL CENTER   
157363                YUMA REGIONAL MEDICAL CENTER   
157364                YUMA REGIONAL MEDICAL CENTER   
157365                YUMA REGIONAL MEDICAL CENTER   
157366                YUMA REGIONAL MEDICAL CENTER   
157367                YUMA REGIONAL MEDICAL CENTER   
157368                YUMA REGIONAL MEDICAL CENTER   
157369                YUMA REGIONAL MEDICAL CENTER   
157370                YUMA REGIONAL MEDICAL CENTER   
157371                YUMA REGIONAL MEDICAL CENTER   
157372  ZUNI COMPREHENSIVE COMMUNITY HEALTH CENTER   

                                                      DRG  Avg_Medicare_Pmts  
0       178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC            8071.08  
1       192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/...            4318.73  
2                  194 - SIMPLE PNEUMONIA & PLEURISY W CC            5298.79  
3            195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC            3534.56  
4                      202 - BRONCHITIS & ASTHMA W CC/MCC            4766.37  
5                       291 - HEART FAILURE & SHOCK W MCC           10678.55  
6                        292 - HEART FAILURE & SHOCK W CC            6097.47  
7                  293 - HEART FAILURE & SHOCK W/O CC/MCC   

# 4. Is there a way to group facilities in a logical way?

You could group them by their top DRGs.

In [23]:
hosp_df.groupby(['DRG', 'Prov_Name'], as_index=False)['Dischg_perc'].agg(['mean'])

mean
DRG                                                Prov_Name                                            
190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC  MARSHALL MEDICAL CENTER SOUTH                0.059901
238 - MAJOR CARDIOVASC PROCEDURES W/O MCC          PHOENIX BAPTIST HOSPITAL                     0.102883
247 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT... BAPTIST MEDICAL CENTER-PRINCETON             0.047720
                                                   CARONDELET HEART AND VASCULAR INSTITUTE      0.139219
287 - CIRCULATORY DISORDERS EXCEPT AMI, W CARD ... SHELBY BAPTIST MEDICAL CENTER                0.047241
                                                   SOUTH BALDWIN REGIONAL MEDICAL CENTER        0.053476
292 - HEART FAILURE & SHOCK W CC                   BANNER HEART HOSPITAL                        0.098780
392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISO... BANNER THUNDERBIRD MEDICAL CENTER            0.050660
                                                   D C H REGIONAL MEDICAL CENTER                0.040278
                                                   MOBILE INFIRMARY                             0.043630
460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC        ST JOSEPH'S HOSPITAL AND MEDICAL CENTER      0.067556
470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT O... ALASKA REGIONAL HOSPITAL                     0.298003
                                                   ARROWHEAD HOSPITAL                           0.115324
                                                   BANNER BOSWELL MEDICAL CENTER                0.074098
                                                   BANNER DEL E WEBB MEDICAL CENTER             0.082064
                                                   BANNER GOOD SAMARITAN MEDICAL CENTER         0.062110
                                                   BAPTIST MEDICAL CENTER SOUTH                 0.058428
                                                   BROOKWOOD MEDICAL CENTER                     0.080175
                                                   CARONDELET ST JOSEPHS HOSPITAL               0.063931
                                                   EAST ALABAMA MEDICAL CENTER AND SNF          0.073159
                                                   ELIZA COFFEE MEMORIAL HOSPITAL               0.054089
                                                   FLAGSTAFF MEDICAL CENTER                     0.132887
                                                   FLOWERS HOSPITAL                             0.087714
                                                   GADSDEN REGIONAL MEDICAL CENTER              0.059982
                                                   HAVASU REGIONAL MEDICAL CENTER               0.124766
                                                   HUNTSVILLE HOSPITAL                          0.078518
                                                   MAYO CLINIC HOSPITAL                         0.112894
                                                   NORTHEAST ALABAMA REGIONAL MED CENTER        0.043264
                                                   PROVIDENCE HOSPITAL                          0.042799
                                                   SCOTTSDALE HEALTHCARE-OSBORN MEDICAL CENTER  0.107973
                                                   SCOTTSDALE HEALTHCARE-SHEA MEDICAL CENTER    0.173549
                                                   ST VINCENT INFIRMARY MEDICAL CENTER          0.085939
                                                   ST VINCENT'S BIRMINGHAM                      0.122383
                                                   ST VINCENT'S EAST                            0.078162
                                                   SUMMIT HEALTHCARE REGIONAL MEDICAL CENTER    0.191987
                                                   THOMAS HOSPITAL                              0.151633
                                                   TRINITY MEDICAL CENTER                       0.066762
     